In [7]:
import cv2 as cv
import numpy as np

import torch
import torchvision
import torchvision.utils
from torchvision.models import detection
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [8]:
device = torch.device("cuda:0")

model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
model.eval()
model = model.float().to(device)

In [9]:
cap = cv.VideoCapture('mems.mp4')

threshold = 0.85
while True:
    ret, frame = cap.read()
    
    if ret:
        frame = torch.tensor(np.swapaxes(np.swapaxes(frame, 0, 2), 1, 2)).float() / 255
        label = model([frame.to(device)])[0]
        frame =  frame.permute(1, 2, 0).detach().cpu().numpy()
        
        if label != None:
            for i in range(len(label["labels"])):
                
                if label["scores"][i] > threshold and label["labels"][i] == 1:
                    frame = cv.rectangle(frame, 
                                        (int(label['boxes'][i][0]), int(label['boxes'][i][1]) ), 
                                        (int(label['boxes'][i][2]), int(label['boxes'][i][3])), 
                                        (255, 255, 0), 1)
        
        cv.imshow('frame', frame)
        if cv.waitKey(15) & 0xFF == ord('q'):
            break
    else:
        cap.set(cv.CAP_PROP_POS_FRAMES, 0)

cap.release()
cv.destroyAllWindows()